In [ ]:
library(sdmpredictors)
library(broom.mixed)
library(ggsignif)
library(lmerTest)
library(multcomp)
library(reshape2)
library(recipes)
library(parsnip)
library(ggpmisc)
library(ggpubr)
library(geodata)
library(ggplot2)
library(stringr)
library(raster)
library(readxl)
library(xtable)
library(broom)
library(dplyr)
library(ade4)
library(caret)
library(ggsci)
library(lme4)
library(nlme)
library(MASS)
library(sf)

In [ ]:
help(sdmpredictors)

In [ ]:
citation("sdmpredictors")

In [ ]:
packageVersion("sdmpredictors")

In [ ]:
(Bluecarbon_sites <- read.csv("data//BC/BCsampling_sites.tsv", sep ="\t", header=T) %>% 
                    dplyr::mutate(Site = ifelse(Site =="Site1", "Upper", "Lower")) %>%
                    dplyr::rename(estuary = Estuary) %>%
                    dplyr::rename(Biogeographical_region = Biogeographical.region))

In [ ]:
sample_n(Bluecarbon_sites, 5)

In [ ]:
CHNS_cores <- readRDS(file = "data/BC/CHNS_cores.RDS") %>%
              dplyr::rename(Plot = plot) %>%
              dplyr::mutate(Site = ifelse(Plot == "LWR", "Lower",  "Upper")) %>%
              dplyr::mutate(Site = as.factor(Site)) %>% 
              dplyr::group_by(estuary, Site) %>%
              dplyr::filter(estuary != "Breede" | Site != "Lower") %>%
              dplyr::mutate(Site_Id = as.factor(paste0("Site", cur_group_id()))) %>%
              dplyr::mutate(Site_ref =  glue::glue("{estuary}_{Site_Id}")) %>%
              ungroup()

estuary_levels = c("Olifants", "Berg","Breede","Knysna", "Swartkops","Mngazana")
CHNS_cores$estuary <- factor(CHNS_cores$estuary, levels = estuary_levels)            

In [ ]:
ZAF_bluecarbon_raw <- CHNS_cores

In [ ]:
xtabs(~ depth + estuary , data = CHNS_cores)

In [ ]:
xtabs(~ estuary + Site, data = CHNS_cores)

In [ ]:
core_count <- ZAF_bluecarbon_raw %>% 
              dplyr::group_by(estuary, Site) %>%
              dplyr::summarise(Cores = n())

BC_samples <- merge(Bluecarbon_sites, core_count) %>%
              dplyr::rename(Long = x, Lat = y) %>%
              dplyr::select(-date) %>%
              dplyr::mutate(estuary = factor(estuary, levels = estuary_levels)) %>%
              dplyr::arrange(estuary, Site)

#print(xtable(BC_samples, digits=7, type = "latex"), file = "Supplementary_material/BC_samples.tex")

In [ ]:
#Follows the Blue carbon manual to calculate total Corg per sediment core
source("bluecarbon_library.R")
core_Corg <-    MgC_cores(CHNS_cores) %>%
                dplyr::select(estuary, Site, Site_Id, Site_ref, Core, mean_Corg, MgC_perHa)

In [ ]:
ZAF_bluecarbon_raw <- merge(core_Corg, Bluecarbon_sites)
xtabs(~estuary + Site,  data  = core_Corg)
colnames(ZAF_bluecarbon_raw)

In [ ]:
dim(core_Corg)
dim(ZAF_bluecarbon_raw)

In [ ]:
#South African Admin shape file
ZA_admin1 <- geodata::gadm("ZA", path = "data/sdm/")
ZA_admin1 = sf::st_as_sf(ZA_admin1)

In [ ]:
ggplot(data = ZA_admin1) +
   geom_sf() +
   geom_point(data = Bluecarbon_sites,
              position = position_dodge(width=0.2),
              aes(x = x,  y = y, colour = Site), size = 2) +
   theme_bw()

In [ ]:
#setting options for smd data
options(sdmpredictors_datadir="C:/Users/andhlovu/Documents/Mzansi-Blue-Carbon/data/sdm")

In [ ]:
sdm_datasets <- list_datasets(terrestrial = TRUE, marine = TRUE)

In [ ]:
sdm_datasets

In [ ]:
sdm_datasets$dataset_code

In [ ]:
environmental_layers <- sdmpredictors::list_layers(sdm_datasets)

In [ ]:
ref_layer = sdmpredictors::list_layers(sdm_datasets)

In [ ]:
colnames(environmental_layers)

In [ ]:
WorldClim_list <- environmental_layers %>% dplyr::filter(is.na(month)) %>% dplyr::filter(dataset_code == "WorldClim") 
MARSPEC_list <- environmental_layers %>% dplyr::filter(is.na(month)) %>% dplyr::filter(dataset_code == "MARSPEC") 

In [ ]:
WorldClim_layers <- sdmpredictors::load_layers(layercodes =  WorldClim_list$layer_code, equalarea=FALSE, rasterstack=TRUE)
MARSPEC_layers <- sdmpredictors::load_layers(layercodes =  MARSPEC_list$layer_code, equalarea=FALSE, rasterstack=TRUE)

In [ ]:
ZA_extent <- raster::extent(10, 40, -37, -22)
WorldClim_cropped <- raster::crop(WorldClim_layers, ZA_extent) 
MARSPEC_cropped <- raster::crop(MARSPEC_layers, ZA_extent)  

In [ ]:
Bluecarbon_sites

In [ ]:
WC_alt_layer <- WorldClim_cropped[[1]]
#env_layers_raw  <- stack(WorldClim_cropped, MARSPEC_cropped)
env_layers <-resample(WorldClim_cropped, WC_alt_layer, method = "ngb") 

In [ ]:
# t <- extent(-180, 180, -90, 90) #layer extent from terrestrial stack
# m <- extent(-180, 180, -90, 90) #layer extent from marine stack
# #no need to edit the following 6 lines
# extent_list<-list(t, m)
# extent_list<-lapply(extent_list, as.matrix)
# matrix_extent<-matrix(unlist(extent_list), ncol=length(extent_list))
# rownames(matrix_extent)<-c("xmin", "ymin", "xmax", "ymax")
# best_extent<-extent(min(matrix_extent[1,]), max(matrix_extent[3,]), min(matrix_extent[2,]), max(matrix_extent[4,]))
# ranges<-apply(as.matrix(best_extent), 1, diff)
# reso <- res(WorldClim_cropped) #choose layer you want to keep resolution
# nrow_ncol <-ranges/reso
# raster_ref <-raster(best_extent, nrows=nrow_ncol[2], ncols=nrow_ncol[1], crs = WorldClim_layers@crs) #choose layer crs you want to keep

In [ ]:
envs_data <- as.data.frame(raster::extract(env_layers, ZAF_bluecarbon_raw[c("x", "y")]))
colnames(envs_data)

In [ ]:
envs_df <- as.data.frame(envs_data)

In [ ]:
pca1 <- dudi.pca(envs_data, 
                 scannf = F, 
                 nf = 2)

In [ ]:
str(pca1$li[,1:2])

In [ ]:
s.corcircle(pca1$co)

In [ ]:
worldclim_ref <- environmental_layers[environmental_layers$layer_code %in% colnames(envs_data), c("layer_code","name","description", "start_year","end_year")]

In [ ]:
colnames(environmental_layers)

In [ ]:
colnames(envs_data)

In [ ]:
worldclim_cor <- layers_correlation(colnames(envs_data))

In [ ]:
high_cor <- findCorrelation(worldclim_cor, cutoff = .6, exact = FALSE, names = TRUE)
#worldclim_ref[worldclim_ref$layer_code %in% high_cor,]

In [ ]:
`%nin%` <- Negate(`%in%`)

In [ ]:
worldclim_best <- worldclim_ref$layer_code[worldclim_ref$layer_code %nin% high_cor]
#worldclim_ref[worldclim_ref$layer_code %in% worldclim_best,]

In [ ]:
worldclim_ref[worldclim_ref$layer_code %in% worldclim_best,]$name

In [ ]:
layers_correlation(worldclim_best)

In [ ]:
colnames(ZAF_bluecarbon_raw)

In [ ]:
ZAF_bluecarbon_final <- data.frame(ZAF_bluecarbon_raw)

for (layer_name in worldclim_best){ 
    
    layer <- env_layers[[layer_name]]
    
    ZAF_bluecarbon_final[layer_name] <- raster::extract(layer, ZAF_bluecarbon_raw[,c("x", "y")])
}

ZAF_bluecarbon_final <- ZAF_bluecarbon_final %>% dplyr::select(-c(WC_alt))

In [ ]:
best_WC <- worldclim_ref[worldclim_ref$layer_code %in% worldclim_best,]

In [ ]:
colnames(ZAF_bluecarbon_final)

In [ ]:
?car::vif

In [ ]:
citation("car")

In [ ]:
fit1 <- lm(MgC_perHa ~ WC_bio2 + WC_bio14 + WC_bio15 + WC_bio18, data = ZAF_bluecarbon_final)
vif_out <-  as.data.frame(car::vif(fit1)) %>% tibble::rownames_to_column(var = "layer_code")
colnames(vif_out)[2] <- "VIF"
(final_layers <- merge(vif_out, best_WC) %>% filter(VIF < 10)) %>% 
                 dplyr::select(layer_code, name, description, description, start_year, end_year, VIF) %>%
                 dplyr::mutate(VIF = round(VIF,2))
print(xtable(final_layers, digits=2, type = "latex"), file = "Supplementary_material/Worldclim_data.tex", include.rownames=FALSE)

In [ ]:
layers_correlation(final_layers$layer_code)

In [ ]:
ZAF_bluecarbon_final <- data.frame(ZAF_bluecarbon_raw)
for (layer_name in final_layers$layer_code){ 
    
    layer <- env_layers[[layer_name]]
    
    ZAF_bluecarbon_final[layer_name] <- raster::extract(layer, ZAF_bluecarbon_raw[,c("x", "y")])
}

ZAF_bluecarbon_final <- ZAF_bluecarbon_final %>% dplyr::select(-c("x", "y"))

In [ ]:
#Without Swartkops estuary
#ZAF_bluecarbon_final <- ZAF_bluecarbon_final %>% 
#                       dplyr::filter(estuary != "Swartkops" | Site != "Lower") 

In [ ]:
ref_numeric <- ZAF_bluecarbon_final %>% 
               dplyr::select_if(is.numeric) %>%
               dplyr::select(-c(mean_Corg, MgC_perHa)) %>%
               colnames()

Scaler <- caret::preProcess(ZAF_bluecarbon_final, method = list(center = ref_numeric, scale = ref_numeric))
ZAF_bluecarbon <- predict(Scaler, ZAF_bluecarbon_final)

In [ ]:
#https://stats.stackexchange.com/questions/59879/logistic-regression-anova-chi-square-test-vs-significance-of-coefficients-ano

In [ ]:
simple_lmer <- function(pred, response_var, random_fmla, data){
    
    fmla_p2 = paste(pred, random_fmla, sep = " + ")
    fmla = as.formula(paste(response_var, fmla_p2, sep = " ~ "))
    lm_fit <-  lmer(fmla, data = data, REML=FALSE)
    lmer_out <- broom::tidy(lm_fit)
    lmer_out$variable <- pred
     
    return(lmer_out)
}

In [ ]:
simple_lmer_anova <- function(pred, response_var, random_fmla, data){
    
    fmla_p2 = paste(pred, random_fmla, sep = " + ")
    fmla = as.formula(paste(response_var, fmla_p2, sep = " ~ "))
    lm_fit <-  lmer(fmla, data = data, REML=FALSE)
    lmer_anova <- car::Anova(lm_fit, test="Chisq") 
    return(lmer_anova)
}

In [ ]:
lmer_out <- lapply(ref_numeric, 
            simple_lmer, 
            response_var = "MgC_perHa",
            random_fmla = "(1|estuary:Site)",
            data = ZAF_bluecarbon_final)

In [ ]:
lmer_anova_out <-  lapply(ref_numeric, 
                   simple_lmer_anova, 
                   response_var = "MgC_perHa",
                   random_fmla = "(1|estuary:Site)",
                   data = ZAF_bluecarbon_final)

In [ ]:
(lmer_df <-  do.call(rbind, lmer_out) %>%
             data.frame() %>%
             dplyr::filter(effect == "fixed" & is.na(group) & term != "(Intercept)") %>%
             dplyr::select(variable, variable, p.value) %>% 
             mutate(signif = signif_annotate(p.value)) %>%
             arrange(variable) %>%
             rename(layer_code = variable))

In [ ]:
lmer_anova_df <-  do.call(rbind, lmer_anova_out) %>%
             data.frame() %>% tibble::rownames_to_column(var = "layer_code")
bioclim_results <- merge(lmer_anova_df, best_WC) %>% 
                   dplyr::select(description, Chisq, Df, Pr..Chisq.) %>% dplyr::rename(p_value = Pr..Chisq.)

In [ ]:
xtabs(~ Biogeographical_region + estuary, data = ZAF_bluecarbon_final)

In [ ]:
ZAF_bluecarbon_final$Biogeographical_region <- factor(ZAF_bluecarbon_final$Biogeographical_region, levels = c('Cool Temperate', 'Warm temperate','Subtropical'))

In [ ]:
ZAF_biogeograpy <- ZAF_bluecarbon_final %>%
                    group_by(Biogeographical_region) %>% 
                    rstatix::get_summary_stats(MgC_perHa) %>% 
                    dplyr::select(Biogeographical_region, n, mean, sd, se) %>%
                    dplyr::rename(n_Biogeography = n, mean_Biogeography = mean, sd_Biogeography = sd,  se_Biogeography = se)

In [ ]:
baseline_bio <- gls(MgC_perHa ~ 1, data = ZAF_bluecarbon_final, method="ML")
random_bio <- lme(MgC_perHa ~ 1, random = ~1|Site_ref, data = ZAF_bluecarbon_final, method="ML")
anova(baseline_bio, random_bio)

In [ ]:
fit_bio <- lmer(MgC_perHa ~  Biogeographical_region + (1|Site_ref),  data  = ZAF_bluecarbon_final)
(Bluecarbon_sites <- car::Anova(fit_bio, test="Chisq") %>%
          tibble::rownames_to_column(var = "description") %>%
          dplyr::rename(p_value = "Pr(>Chisq)"))

In [ ]:
(env_results <- rbind(Bluecarbon_sites, bioclim_results) %>% 
               dplyr::mutate(p_value = round(p_value, 4), Chisq = round(Chisq, 2)))
#print(xtable(env_results, digits=2), type = "html", file = "Table/env_stats.html", include.rownames=FALSE)
print(xtable(env_results, digits=7, type = "latex"), file = "Supplementary_material/env_stats.tex", include.rownames=FALSE)

In [ ]:
dim(ZAF_bluecarbon_final)

In [ ]:
glht_bio <- glht(fit_bio, linfct = mcp(Biogeographical_region = "Tukey"))
cld_bio <- multcomp::cld(glht_bio)
cld_bio <- cld_bio$mcletters$Letters %>% 
                 data.frame() %>%
                 setNames("cld_Biogeographical_region") %>%
                 tibble::rownames_to_column(var = "Biogeographical_region")

In [ ]:
(table_biogeography1 <- merge(ZAF_biogeograpy, cld_bio) %>%
                       dplyr::mutate(n_Biogeography = glue::glue("({n_Biogeography})")))

In [ ]:
anno_biogeography1 <-  ZAF_bluecarbon_final %>% 
                       dplyr::group_by(Biogeographical_region) %>%
                       dplyr::summarise(y1 = max(MgC_perHa) + 15,  y2 = y1 + 20) %>%
                       dplyr::ungroup() %>%
                       dplyr::mutate(x = row_number())

(biogeography_anno1 <-  merge(anno_biogeography1, table_biogeography1))

In [ ]:
p1 <- ggplot(data = ZAF_bluecarbon_final %>% 
             dplyr::rename(Estuary = estuary, `Sampling site` = Site)) +
      geom_boxplot(aes(y = MgC_perHa,
                      x = Biogeographical_region),
                      outlier.alpha = 0,
                      linewidth = 1) +
     geom_point(aes(y = MgC_perHa,
                x = Biogeographical_region,
                colour = Estuary,
                shape = `Sampling site`,
                fill = Estuary),
                size = 4,
                alpha = 0.5,
               position = "jitter") +
      labs(x  = "Biogeographical region",  y = expression(paste(C[org]~stocks~(Mg~C~ha^{-1})))) +
      scale_y_continuous(limits = c(0, 267), breaks = seq(0, 267, 25)) +
      theme(panel.grid.minor = element_blank(),
            panel.background = element_blank(),
            legend.box.background = element_rect(colour = "black"),
            legend.title = element_text(size=22, colour = "black"),
            legend.text = element_text(size=22, colour = "black"),
            axis.text.y = element_text(size=22, colour = "#434343"),
            axis.text.x = element_text(hjust = 1, size=22, colour = "#434343", angle = 45),
            axis.title = element_text(size=22, colour = "black"),
            panel.border = element_rect(colour = "black", fill=NA, linewidth=1)) +
      geom_text(data = biogeography_anno1,
               aes(label = n_Biogeography,
                   y = y1,
                   x = x),
                   size = 8,
                   alpha = 0.75,
                   colour = "#808080") +
      geom_text(data = biogeography_anno1,
               aes(label = cld_Biogeographical_region,
                   y = y2,
                   x = x),
                   size = 8,
                   colour = "#808080") +
      ggsci::scale_color_d3()

p1
ggsave("Figurex/biogeo.pdf", height = 10,  width = 7.5)

In [ ]:
######################################################################################################

                                  ####  REDUCED DATASET ####

######################################################################################################

In [ ]:
#Without Swartkops estuary
ZAF_bluecarbon_final2 <- ZAF_bluecarbon_final %>% 
                       dplyr::filter(estuary != "Swartkops" | Site != "Lower") 

In [ ]:
ZAF_biogeograpy2 <- ZAF_bluecarbon_final2 %>%
                    group_by(Biogeographical_region) %>% 
                    rstatix::get_summary_stats(MgC_perHa) %>% 
                    dplyr::select(Biogeographical_region, n, mean, sd, se) %>%
                    dplyr::rename(n_Biogeography = n, mean_Biogeography = mean, sd_Biogeography = sd,  se_Biogeography = se)

In [ ]:
ref_numeric <- ZAF_bluecarbon_final %>% 
               dplyr::select_if(is.numeric) %>%
               dplyr::select(-c(mean_Corg, MgC_perHa)) %>%
               colnames()

Scaler <- caret::preProcess(ZAF_bluecarbon_final2, method = list(center = ref_numeric, scale = ref_numeric))
ZAF_bluecarbon <- predict(Scaler, ZAF_bluecarbon_final2)

In [ ]:
#https://stats.stackexchange.com/questions/59879/logistic-regression-anova-chi-square-test-vs-significance-of-coefficients-ano

In [ ]:
simple_lmer <- function(pred, response_var, random_fmla, data){
    
    fmla_p2 = paste(pred, random_fmla, sep = " + ")
    fmla = as.formula(paste(response_var, fmla_p2, sep = " ~ "))
    lm_fit <-  lmer(fmla, data = data, REML=FALSE)
    lmer_out <- broom::tidy(lm_fit)
    lmer_out$variable <- pred
     
    return(lmer_out)
}

In [ ]:
simple_lmer_anova <- function(pred, response_var, random_fmla, data){
    
    fmla_p2 = paste(pred, random_fmla, sep = " + ")
    fmla = as.formula(paste(response_var, fmla_p2, sep = " ~ "))
    lm_fit <-  lmer(fmla, data = data, REML=FALSE)
    lmer_anova <- car::Anova(lm_fit, test="Chisq") 
    return(lmer_anova)
}

In [ ]:
lmer_out <- lapply(ref_numeric, 
            simple_lmer, 
            response_var = "MgC_perHa",
            random_fmla = "(1|estuary:Site)",
            data = ZAF_bluecarbon_final2)

In [ ]:
lmer_anova_out <-  lapply(ref_numeric, 
                   simple_lmer_anova, 
                   response_var = "MgC_perHa",
                   random_fmla = "(1|estuary:Site)",
                   data = ZAF_bluecarbon_final)

In [ ]:
(lmer_df <-  do.call(rbind, lmer_out) %>%
             data.frame() %>%
             dplyr::filter(effect == "fixed" & is.na(group) & term != "(Intercept)") %>%
             dplyr::select(variable, variable, p.value) %>% 
             mutate(signif = signif_annotate(p.value)) %>%
             arrange(variable) %>%
             rename(layer_code = variable))

In [ ]:
lmer_anova_df <-  do.call(rbind, lmer_anova_out) %>%
             data.frame() %>% tibble::rownames_to_column(var = "layer_code")
bioclim_results <- merge(lmer_anova_df, best_WC) %>% 
                   dplyr::select(description, Chisq, Df, Pr..Chisq.) %>% dplyr::rename(p_value = Pr..Chisq.)

In [ ]:
baseline_bio <- gls(MgC_perHa ~ 1, data = ZAF_bluecarbon_final, method="ML")
random_bio <- lme(MgC_perHa ~ 1, random = ~1|Site_ref, data = ZAF_bluecarbon_final, method="ML")
anova(baseline_bio, random_bio)

In [ ]:
fit_bio <- lmer(MgC_perHa ~  Biogeographical_region + (1|Site_ref),  data  = ZAF_bluecarbon_final2)
(Bluecarbon_sites <- car::Anova(fit_bio, test="Chisq") %>%
          tibble::rownames_to_column(var = "description") %>%
          dplyr::rename(p_value = "Pr(>Chisq)"))

In [ ]:
(env_results <- rbind(Bluecarbon_sites, bioclim_results) %>% 
               dplyr::mutate(p_value = round(p_value, 4), Chisq = round(Chisq, 2)))
#print(xtable(env_results, digits=2), type = "html", file = "Table/env_stats.html", include.rownames=FALSE)
print(xtable(env_results, digits=7, type = "latex"), file = "Supplementary_material/env_stats.tex", include.rownames=FALSE)

In [ ]:
glht_bio <- glht(fit_bio, linfct = mcp(Biogeographical_region = "Tukey"))
cld_bio <- multcomp::cld(glht_bio)
cld_bio <- cld_bio$mcletters$Letters %>% 
                 data.frame() %>%
                 setNames("cld_Biogeographical_region") %>%
                 tibble::rownames_to_column(var = "Biogeographical_region")

In [ ]:
table_biogeography2 <- merge(ZAF_biogeograpy2, cld_bio) %>%
                       dplyr::mutate(n_Biogeography = glue::glue("({n_Biogeography})"))

In [ ]:
anno_biogeography2 <-   ZAF_bluecarbon_final2 %>% 
                        dplyr::group_by(Biogeographical_region) %>%
                        dplyr::summarise(y1 = max(MgC_perHa) + 10,  y2 = y1 + 10) %>%
                        dplyr::ungroup() %>%
                        dplyr::mutate(x = row_number())

(biogeography_anno2 <-  merge(anno_biogeography2, table_biogeography2))

In [ ]:
biogeography_anno1

In [ ]:
p2 <- ggplot(data = ZAF_bluecarbon_final2 %>% dplyr::rename(Estuary = estuary)) +
      geom_boxplot(aes(y = MgC_perHa,
                      x = Biogeographical_region),
                      outlier.alpha = 0,
                      linewidth = 1) +
     geom_point(aes(y = MgC_perHa,
                x = Biogeographical_region,
                colour = Estuary,
                fill = Estuary),
                size = 4,
                alpha = 0.5,
               position = "jitter") +
      labs(x  = "Biogeographical region",  y = expression(paste(C[org]~stocks~(Mg~C~ha^{-1})))) +
      scale_y_continuous(limits = c(0, 150), breaks = seq(0, 125, 25)) +
      theme(panel.grid.minor = element_blank(),
            panel.background = element_blank(),
            legend.box.background = element_rect(colour = "black"),
            legend.title = element_text(size=22, colour = "black"),
            legend.text = element_text(size=22, colour = "black"),
            axis.text.y = element_text(size=22, colour = "#434343"),
            axis.text.x = element_text(hjust = 1, size=22, colour = "#434343", angle = 45),
            axis.title = element_text(size=22, colour = "black"),
            panel.border = element_rect(colour = "black", fill=NA, linewidth=1)) +
      geom_text(data = biogeography_anno2,
               aes(label = n_Biogeography,
                   y = y1,
                   x = x),
                   size = 8,
                   alpha = 0.75,
                   colour = "#808080") +
      geom_text(data = biogeography_anno2,
               aes(label = cld_Biogeographical_region,
                   y = y2,
                   x = x),
                   size = 8,
                   colour = "#808080") +
      ggsci::scale_color_d3()

In [ ]:
p2

In [ ]:
p1

In [ ]:
fit_bio2 <- lmer(MgC_perHa ~  Biogeographical_region + (1|Site_ref),  data  = ZAF_bluecarbon_final2)
summary(fit_bio2)